In [44]:
from data_aug.bbox_util import *
import pandas as pd 
import glob
import xml.etree.ElementTree as Xet
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [45]:
path="C:\\Users\Lenovo\Desktop\\"
filenames=glob.glob(path+'*.xml')

In [52]:
def crop(a,bboxes,img,Cropped):
    # a: the crop box 
    cols = ["ID","x1", "y1", "x2", "y2", "c"]
    for i in bboxes:
        #Copy the boundary box params
        x1=i[0]
        y1=i[1]
        x2=i[2]
        y2=i[3]
        c=i[4]

        #Set the margin for the crop  
        x_margin=a-(x2-x1)
        y_margin=a-(y1-y2)
        #Calculate the copping coordinates
        x_1=x1-x_margin//2
        x_2=x_1+a
        y_2=y2-y_margin//2
        y_1=y_2+a
        #Cropping
        print(x_1,x_2,y_1,y_2)
        imag=img[y_2:y_1,x_1:x_2,:]
        #Update the boundary box coordinates
        x1=x1-x_1
        y1=y1-y_2
        x2=x2-x_1
        y2=y2-y_2
        
        rows = []
        ID='gh'+str(i)+'.jpg'
        rows.append({"ID": ID,
                 "x1": x1,
                 "y1": y1,
                 "x2": x2,
                 "y2": y2,
                 "c":1})

        line = pd.DataFrame(rows, columns=cols)
        Cropped=pd.concat([Cropped,line],ignore_index=True)
        #Needs more code XD
        bb=np.array([x1,y1,x2,y2,0])
        plotted_img = draw_rect(imag, bb)
        print(bb)
        save_path='gh'+str(i)+'.jpg'
        plt.imsave(save_path,plotted_img)
        
    return(Cropped)

In [54]:
Cropped=pd.DataFrame()
for file in filenames:
    xmlparse = Xet.parse(file)
    root = xmlparse.getroot()
    bbs=[]
    for i in root:
        if i.find("bndbox/xmin")!=None:
        
            x1= int(i.find("bndbox/xmin").text)
            y1 = int(i.find("bndbox/ymax").text)
            x2 = int(i.find("bndbox/xmax").text)
            y2 = int(i.find("bndbox/ymin").text)
            c=0
            bbs.append([x1,y1,x2,y2,c])
    bbs=np.array(bbs)
    img=cv2.imread(file[:-3]+'JPG')[:,:,::-1]  
    Cropped=crop(300,bbs,img,Cropped)
Cropped.set_index("ID",inplace=True)
Cropped.to_csv('bbox_cropped.csv')    


1149 1449 2681 2381
[ 13 284 287  15   0]
1942 2242 3200 2900
[  2 300 298   0   0]
3064 3364 2332 2032
[ 21 277 278  23   0]
3063 3363 3423 3123
[ 15 288 284  12   0]
3100 3400 4439 4139
[ 13 290 287  10   0]
2141 2441 4371 4071
[ 14 285 286  14   0]
961 1261 4346 4046
[ 30 276 270  23   0]
